In [ ]:
import sente
from go_attack.board_utils import mirror_move

move = sente.Move(0, 0, sente.stone.BLACK)
mirrored = mirror_move(move)
mirrored2 = mirror_move(mirrored)

In [ ]:
mirrored, mirrored2

In [ ]:
from pathlib import Path
import sente

def load_games(path):
    games = [
        sente.sgf.load(str(path))
        for path in Path(path).glob('*.sgf')
    ]

    for game in games:
        game.play_default_sequence()
        if not game.is_over():
            game.play(None)
            assert game.is_over()
    
    return games

def compute_win_margins(games, black = True):
    victim = sente.stone.BLACK if black else sente.stone.WHITE
    attacker = sente.stone.WHITE if black else sente.stone.BLACK
    return [
        game.score()[victim] - game.score()[attacker]
        for game in games
    ]

In [ ]:
def mirror_move(opponent: sente.Move, board_size: int = 19) -> sente.Move:
    last = board_size - 1
    opponent_x, opponent_y = opponent.get_x(), opponent.get_y()

    # Did they play on the y = x diagonal?
    # If so, mirror across the y = -x diagonal.
    if opponent_x == last - opponent_y:
        # Mirror across the y = -x diagonal
        print("Mirroring across the y = -x diagonal")
        mirror_x = opponent_y
        mirror_y = opponent_x

    # Normal case: mirror across the y = x diagonal
    else:
        print("Mirroring across the y = x diagonal")
        mirror_x = last - opponent_x
        mirror_y = last - opponent_y

    black = opponent.get_stone() == sente.stone.BLACK
    return sente.Move(
        mirror_x,
        mirror_y,
        sente.stone.WHITE if black else sente.stone.BLACK
    )


mirror_move(sente.Move(1, 1, sente.stone.BLACK))

In [ ]:
from collections import defaultdict
import sente

games = {}
folders = (
    "edges-black", "edges-white", "pass-black", "pass-white",
    "random-black", "random-white"
)
for folder in folders:
    games[folder] = load_games(Path.home() / folder)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(2, 3)
fig.set_size_inches(18, 9)

for i, strat in enumerate(("Edges", "Pass", "Random")):
    black = compute_win_margins(games[f'{strat.lower()}-black'], black=True)
    white = compute_win_margins(games[f'{strat.lower()}-white'] , black=False)

    if strat == "Edges":
        black_wins = sum(black > 0 for black in black)
        white_wins = sum(white > 0 for white in white)
        black_losses = sum(black < 0 for black in black)
        white_losses = sum(white < 0 for white in white)
        black_draws = sum(black == 0 for black in black)
        white_draws = sum(white == 0 for white in white)

        ax[0, i].text(100, 110, f"Wins: {black_wins} ({100 * black_wins / 250:.1f}%) Losses: {black_losses} ({100 * black_losses / 250:.1f}%)")
        ax[0, i].text(100, 90, "Worst score: " + str(min(black)))
    
    ax[0, i].hist(black)
    ax[1, i].hist(white)
    ax[0, i].set_title(f'{strat} Strategy (Black Victim)')
    ax[1, i].set_title(f'{strat} Strategy (White Victim)')

fig.suptitle("KataGo Win Margin Against Baseline Adversaries")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(12, 6)

black = [
    game.score()[sente.stone.BLACK] - game.score()[sente.stone.WHITE]
    for game in games[f'edges-black']
]
white = [
    game.score()[sente.stone.WHITE] - game.score()[sente.stone.BLACK]
    for game in games[f'edges-white']
]
black_length = [len(game.get_current_sequence()) for game in games[f'edges-black']]
white_length = [len(game.get_current_sequence()) for game in games[f'edges-white']]
losing_lengths = [l for l, s in zip(black_length, black) if s < 0]
winning_lengths = [l for l, s in zip(black_length, black) if s > 0]

ax[0].scatter(black_length, black)
ax[1].scatter(white_length, white)
ax[0].text(290, 250, f"Max Len. of Losing Game: {max(losing_lengths)}")
ax[0].text(290, 225, f"Median Len. of Losing Game: {np.median(losing_lengths):.0f}")
ax[0].text(290, 200, f"Median Len. of Winning Game: {np.median(winning_lengths):.0f}")
ax[0].set_title('Edges Strategy (Black Victim)')
ax[1].set_title('Edges Strategy (White Victim)')
ax[0].set_xlabel('Length of Game')
ax[1].set_xlabel('Length of Game')

fig.suptitle("KataGo Adversarial Win Margin vs. Length of Game")

In [ ]:
losing_games = [
    game
    for game in games[f'edges-black']
    if game.score()[sente.stone.BLACK] - game.score()[sente.stone.WHITE] < 0
]
worst_game = min(losing_games, key=lambda g: g.score()[sente.stone.BLACK] - g.score()[sente.stone.WHITE])

In [ ]:
sente.sgf.dump(worst_game, str(Path.home() / "losing-game.sgf"))

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

games_1600_hard = load_games(Path.home() / "edges-black-1600-hard")

In [ ]:
plt.hist(compute_win_margins(games_1600_hard))

In [ ]:
games_1600 = load_games(Path.home() / "edges-black-1600")

In [ ]:
plt.hist(compute_win_margins(games_1600))

In [ ]:
games_hard = load_games(Path.home() / "edges-black-512-hard")

In [ ]:
games_normal = load_games(Path.home() / "edges-black")

In [ ]:
plt.hist(compute_win_margins(games_normal))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(2, 2)
fig.set_size_inches(18, 9)

configs = (
    ("Small Net, 512 Playouts", games_normal),
    ("Small Net, 1600 Playouts", games_1600),
    ("Large Net, 512 Playouts", games_hard),
    ("Large Net, 1600 Playouts", games_1600_hard)
)
for i, (title, g) in enumerate(configs):
    margins = compute_win_margins(g, black=True)
    ax[i // 2, i % 2].hist(margins)
    ax[i // 2, i % 2].set_title(title)

    wins = sum(s > 0 for s in margins)
    losses = sum(s < 0 for s in margins)
    draws = sum(s == 0 for s in margins)

    if losses:
        ax[i // 2, i % 2].text(100, 110, f"Wins: {wins} ({100 * wins / 250:.1f}%) Losses: {losses} ({100 * losses / 250:.1f}%)")
        ax[i // 2, i % 2].text(100, 90, "Worst score: " + str(min(margins)))

fig.suptitle("KataGo Win Margin Against Baseline Adversaries")

In [ ]:
games_normal